In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [2]:
from helpers import load_csv_data

DATA_TRAIN_PATH = 'data/train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


## Prepare cost and prediction functions

In [5]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    hinge_loss = np.maximum(0, (1 - y * (X @ w)))
    return np.sum(hinge_loss) + lambda_ / 2 * w.T @ w

In [6]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    y_pred = np.where(X @ w > 0, 1, -1)
    return np.mean(y == y_pred)

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [26]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    # x_n, shape = (num_features,)
    # y_n, shape = ()
    x_n, y_n = X[n], y[n]
    # subgradient (if (1 - yxw < 0) --> ft should be 0)
    ft = - y_n * x_n.T if y_n * (x_n @ w) < 1 else np.zeros(x_n.T.shape)
    ft = np.squeeze(ft) * num_examples
    st = lambda_ * w
    return ft + st

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [29]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 1
    lambda_ = 0.01
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0, num_examples - 1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma / (it + 1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration = {i}, cost = {c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration = 0, cost = 35816485958556.91
iteration = 10000, cost = 22922290382.058964
iteration = 20000, cost = 18896833461.223423
iteration = 30000, cost = 18065060972.421722
iteration = 40000, cost = 15493871857.709267
iteration = 50000, cost = 14530487268.980503
iteration = 60000, cost = 13891201043.224844
iteration = 70000, cost = 13247308420.101677
iteration = 80000, cost = 13321051509.99869
iteration = 90000, cost = 12110896724.795107
training accuracy = 0.7002


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [33]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    lambda_: regularization parameter
    alpha: dual variable, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    grad = y_n * (x_n @ w) - 1
    
    if old_alpha_n == 0:
        p_grad = min(grad, 0)
    elif old_alpha_n == 1:
        p_grad = max(grad, 0)
    else:
        p_grad = grad
        
    if p_grad != 0:
        alpha[n] = min(max(old_alpha_n - lambda_ * grad / (x_n.T @ x_n), 0), 1)
        w += 1 / lambda_ * (alpha[n] - old_alpha_n) * y_n * x_n
    
    return w, alpha

In [34]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    return np.sum(alpha) - lambda_ / 2 * w.T @ w

In [35]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 100000
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:3628.84256, dual:0.00000, gap:3628.84256
iteration=10000, primal:6778.19830, dual:0.00009, gap:6778.19821
iteration=20000, primal:3890.89915, dual:0.00018, gap:3890.89897
iteration=30000, primal:4889.72173, dual:0.00027, gap:4889.72147
iteration=40000, primal:6089.10069, dual:0.00035, gap:6089.10034
iteration=50000, primal:4627.37154, dual:0.00043, gap:4627.37111
iteration=60000, primal:5802.57802, dual:0.00052, gap:5802.57750
iteration=70000, primal:3610.92103, dual:0.00061, gap:3610.92043
iteration=80000, primal:6094.18372, dual:0.00069, gap:6094.18303
iteration=90000, primal:3261.59202, dual:0.00078, gap:3261.59124
training accuracy = 0.6968
